# Supervised Learning with scikit learn

## Table of contents:

* <a href=#Class>Classification</a>
* <a href=#Regres>Regression</a>
* <a href=#Tuning>Fine-tuning your model</a>
* <a href=#Pipe>Preprocessing and pipelines</a>

## Load Packages and Set Global Variables

<a id="imports"></a>

In [21]:
import numpy as np
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV

from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error, classification_report, confusion_matrix, roc_curve, roc_auc_score
from sklearn.preprocessing import Imputer, scale, StandardScaler
from sklearn.pipeline import Pipeline

from scipy.stats import randint
from sklearn.tree import DecisionTreeClassifier 
from sklearn.svm import SVC







from sklearn import datasets
import matplotlib.pyplot as plt

## Global Variables

All embeddings and clusterings can be saved and loaded into this script. Be carful with overwriting cluster caches as soon as cell type annotation has started as cluster labels may be shuffled.

Set whether anndata objects are recomputed or loaded from cache.

In [5]:
bool_recomp = False

Set whether clustering is recomputed or loaded from saved .obs file. Loading makes sense if the clustering changes due to a change in scanpy or one of its dependencies and the number of clusters or the cluster labels change accordingly.

In [6]:
bool_recluster = False

Set whether cluster cache is overwritten. Note that the cache exists for reproducibility of clustering, see above.

In [7]:
bool_write_cluster_cache = False

Set whether to produce plots, set to False for test runs.

In [8]:
bool_plot = False

Set whether observations should be calculated. If false, it is necessary to read cacheed file that contains the necssary information. It then shows the the distributions of counts and genes, as well as mt_frac after filtering. 
Set to true in order to see the data before filtering and follow the decisions for cutoffs.

In [9]:
bool_create_observations = True

<a id="Dataloading"></a>

## Classification

Create an instance of a k-NN classifier with 6 neighbors (by specifying the n_neighbors parameter) and then fit it to the data.

In [10]:
if bool_recomp == True:   
    # Import KNeighborsClassifier from sklearn.neighbors

    # Create arrays for the features and the response variable
    y = df['party'].values
    X = df.drop('party', axis=1).values

    # Create a k-NN classifier with 6 neighbors
    knn = KNeighborsClassifier(n_neighbors=6)

    # Fit the classifier to the data
    knn.fit(X,y)

Having fit a k-NN classifier, you can now use it to predict the label of a new data point. 

In [11]:
if bool_recomp == True:    
    # Import KNeighborsClassifier from sklearn.neighbors

    # Create arrays for the features and the response variable
    y = df['party'].values
    X = df.drop(['party'], axis=1).values

    # Create a k-NN classifier with 6 neighbors: knn
    knn = KNeighborsClassifier(n_neighbors=6)

    # Fit the classifier to the data
    knn.fit(X,y)

    # Predict the labels for the training data X
    y_pred = knn.predict(X)

    # Predict and print the label for the new data point X_new
    new_prediction = knn.predict(X_new)
    print("Prediction: {}".format(new_prediction))

Multi-class classification with the MNIST digits recognition dataset.

In [14]:
if bool_recomp == True:
    # Load the digits dataset: digits
    digits = datasets.load_digits()

    # Print the keys and DESCR of the dataset
    print(digits.keys())
    print(digits.DESCR)

    # Print the shape of the images and data keys
    print(digits.images.shape)
    print(digits.data.shape)

    # Display digit 1010
    plt.imshow(digits.images[1010], cmap=plt.cm.gray_r, interpolation='nearest')
    plt.show()

Train/Test Split + Fit/Predict/Accuracy

In [17]:
if bool_recomp == True:
    # Create feature and target arrays
    X = digits.data
    y = digits.target

    # Split into training and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42, stratify=y)

    # Create a k-NN classifier with 7 neighbors: knn
    knn = KNeighborsClassifier(n_neighbors=7)

    # Fit the classifier to the training data
    knn.fit(X_train, y_train)

    # Print the accuracy
    print(knn.score(X_test, y_test))

Overfitting and underfitting

In [ ]:
if bool_recomp == True:
    # Setup arrays to store train and test accuracies
    neighbors = np.arange(1, 9)
    train_accuracy = np.empty(len(neighbors))
    test_accuracy = np.empty(len(neighbors))

    # Loop over different values of k
    for i, k in enumerate(neighbors):
        # Setup a k-NN Classifier with k neighbors: knn
        knn = KNeighborsClassifier(n_neighbors=k)

        # Fit the classifier to the training data
        knn.fit(X_train, y_train)

        #Compute accuracy on the training set
        train_accuracy[i] = knn.score(X_train, y_train)

        #Compute accuracy on the testing set
        test_accuracy[i] = knn.score(X_test, y_test)

    # Generate plot
    plt.title('k-NN: Varying Number of Neighbors')
    plt.plot(neighbors, test_accuracy, label = 'Testing Accuracy')
    plt.plot(neighbors, train_accuracy, label = 'Training Accuracy')
    plt.legend()
    plt.xlabel('Number of Neighbors')
    plt.ylabel('Accuracy')
    plt.show()

## Regression

Import the data for supervised learning and get it into the form needed by scikit-learn

In [ ]:
if bool_recomp == True:
    # Read the CSV file into a DataFrame: df
    df = pd.read_csv('gapminder.csv')

    # Create arrays for features and target variable
    y = df['life'].values
    X = df['fertility'].values

    # Print the dimensions of y and X before reshaping
    print("Dimensions of y before reshaping: ", y.shape)
    print("Dimensions of X before reshaping: ", X.shape)

    # Reshape X and y
    y_reshaped = y.reshape(-1,1)
    X_reshaped = X.reshape(-1,1)

    # Print the dimensions of y_reshaped and X_reshaped
    print("Dimensions of y after reshaping: ", y_reshaped.shape)
    print("Dimensions of X after reshaping: ", X_reshaped.shape)

Fit & predict for regression

In [18]:
if bool_recomp == True:
    # Create the regressor: reg
    reg = LinearRegression()

    # Create the prediction space
    prediction_space = np.linspace(min(X_fertility), max(X_fertility)).reshape(-1,1)

    # Fit the model to the data
    reg.fit(X_fertility, y)

    # Compute predictions over the prediction space: y_pred
    y_pred = reg.predict(prediction_space)

    # Print R^2 
    print(reg.score(X_fertility, y))

    # Plot regression line
    plt.plot(prediction_space, y_pred, color='black', linewidth=3)
    plt.show()

Train/test split for regression

In [ ]:
if bool_recomp == True:    
    # Create training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

    # Create the regressor: reg_all
    reg_all = LinearRegression()

    # Fit the regressor to the training data
    reg_all.fit(X_train, y_train)

    # Predict on the test data: y_pred
    y_pred = reg_all.predict(X_test)

    # Compute and print R^2 and RMSE
    print("R^2: {}".format(reg_all.score(X_test, y_test)))
    rmse = np.sqrt(mean_squared_error(y_test,y_pred))
    print("Root Mean Squared Error: {}".format(rmse))

5-fold cross-validation

In [ ]:
if bool_recomp == True:    
    # Create a linear regression object: reg
    reg = LinearRegression()

    # Compute 5-fold cross-validation scores: cv_scores
    cv_scores = cross_val_score(reg, X, y, cv=5)

    # Print the 5-fold cross-validation scores
    print(cv_scores)

    print("Average 5-Fold CV Score: {}".format(np.mean(cv_scores)))

K-Fold CV comparison

In [ ]:
if bool_recomp == True:    
    # Create a linear regression object: reg
    reg = LinearRegression()

    # Perform 3-fold CV
    cvscores_3 = cross_val_score(reg, X, y, cv=3)
    print(np.mean(cvscores_3))

    # Perform 10-fold CV
    cvscores_10 = cross_val_score(reg, X, y, cv=10)
    print(np.mean(cvscores_10))

Lasso regularization 

In [19]:
if bool_recomp == True:     
    # Instantiate a lasso regressor: lasso
    lasso = Lasso(alpha=0.4, normalize=True)

    # Fit the regressor to the data
    lasso.fit(X, y)

    # Compute and print the coefficients
    lasso_coef = lasso.fit(X,y).coef_
    print(lasso_coef)

    # Plot the coefficients
    plt.plot(range(len(df_columns)), lasso_coef)
    plt.xticks(range(len(df_columns)), df_columns.values, rotation=60)
    plt.margins(0.02)
    plt.show()

Ridge regularization

In [ ]:
if bool_recomp == True: 
    
    def display_plot(cv_scores, cv_scores_std):
        fig = plt.figure()
        ax = fig.add_subplot(1,1,1)
        ax.plot(alpha_space, cv_scores)

        std_error = cv_scores_std / np.sqrt(10)

        ax.fill_between(alpha_space, cv_scores + std_error, cv_scores - std_error, alpha=0.2)
        ax.set_ylabel('CV Score +/- Std Error')
        ax.set_xlabel('Alpha')
        ax.axhline(np.max(cv_scores), linestyle='--', color='.5')
        ax.set_xlim([alpha_space[0], alpha_space[-1]])
        ax.set_xscale('log')
        plt.show()
    
    # Setup the array of alphas and lists to store scores
    alpha_space = np.logspace(-4, 0, 50)
    ridge_scores = []
    ridge_scores_std = []

    # Create a ridge regressor: ridge
    ridge = Ridge(normalize=True)

    # Compute scores over range of alphas
    for alpha in alpha_space:

        # Specify the alpha value to use: ridge.alpha
        ridge.alpha = alpha

        # Perform 10-fold CV: ridge_cv_scores
        ridge_cv_scores = cross_val_score(ridge, X, y, cv=10)

        # Append the mean of ridge_cv_scores to ridge_scores
        ridge_scores.append(np.mean(ridge_cv_scores))

        # Append the std of ridge_cv_scores to ridge_scores_std
        ridge_scores_std.append(np.std(ridge_cv_scores))

    # Display the plot
    display_plot(ridge_scores, ridge_scores_std)


## Fine-tunning your model

Metrics for classification - train a k-NN classifier to the data and evaluate its performance by generating a confusion matrix and classification report.

In [20]:
if bool_recomp == True:     
    # Create training and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

    # Instantiate a k-NN classifier: knn
    knn = KNeighborsClassifier(n_neighbors=6)

    # Fit the classifier to the training data
    knn.fit(X_train, y_train)

    # Predict the labels of the test data: y_pred
    y_pred = knn.predict(X_test)

    # Generate the confusion matrix and classification report
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

Building a logistic regression model

In [25]:
if bool_recomp == True:  
    # Create training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state=42)

    # Create the classifier: logreg
    logreg = LogisticRegression()

    # Fit the classifier to the training data
    logreg.fit(X_train, y_train)

    # Predict the labels of the test set: y_pred
    y_pred = logreg.predict(X_test)

    # Compute and print the confusion matrix and classification report
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

Plotting an ROC curve

In [26]:
if bool_recomp == True: 
    # Compute predicted probabilities: y_pred_prob
    y_pred_prob = logreg.predict_proba(X_test)[:,1]

    # Generate ROC curve values: fpr, tpr, thresholds
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

    # Plot ROC curve
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr, tpr, label='Logistiic Regression')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.show()

AUC computation

In [27]:
if bool_recomp == True: 
    # Compute predicted probabilities: y_pred_prob
    y_pred_prob = logreg.predict_proba(X_test)[:,1]

    # Compute and print AUC score
    print("AUC: {}".format(roc_auc_score(y_test, y_pred_prob)))

    # Compute cross-validated AUC scores: cv_auc
    cv_auc = cross_val_score(logreg, X, y, cv=5, scoring='roc_auc')

    # Print list of AUC scores
    print("AUC scores computed using 5-fold cross-validation: {}".format(cv_auc))

Hyperparameter tuning with GridSearchCV - focus on the process of setting up the hyperparameter grid and performing grid-search cross-validation

In [28]:
if bool_recomp == True: 
    # Setup the hyperparameter grid
    c_space = np.logspace(-5, 8, 15)
    param_grid = {'C': c_space}

    # Instantiate a logistic regression classifier: logreg
    logreg = LogisticRegression()

    # Instantiate the GridSearchCV object: logreg_cv
    logreg_cv = GridSearchCV(logreg, param_grid, cv=5)

    # Fit it to the data
    logreg_cv.fit(X,y)

    # Print the tuned parameters and score
    print("Tuned Logistic Regression Parameters: {}".format(logreg_cv.best_params_)) 
    print("Best score is {}".format(logreg_cv.best_score_))

Hyperparameter tuning with RandomizedSearchCV

In [29]:
if bool_recomp == True: 
    # Setup the parameters and distributions to sample from: param_dist
    param_dist = {"max_depth": [3, None],
                  "max_features": randint(1, 9),
                  "min_samples_leaf": randint(1, 9),
                  "criterion": ["gini", "entropy"]}

    # Instantiate a Decision Tree classifier: tree
    tree = DecisionTreeClassifier()

    # Instantiate the RandomizedSearchCV object: tree_cv
    tree_cv = RandomizedSearchCV(tree, param_dist, cv=5)

    # Fit it to the data
    tree_cv.fit(X, y)

    # Print the tuned parameters and score
    print("Tuned Decision Tree Parameters: {}".format(tree_cv.best_params_))
    print("Best score is {}".format(tree_cv.best_score_))

Hold-out set in practice I: Classification

In [30]:
if bool_recomp == True: 
    # Create the hyperparameter grid
    c_space = np.logspace(-5, 8, 15)
    param_grid = {'C': c_space, 'penalty': ['l1', 'l2']}

    # Instantiate the logistic regression classifier: logreg
    logreg = LogisticRegression()

    # Create train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state=42)

    # Instantiate the GridSearchCV object: logreg_cv
    logreg_cv = GridSearchCV(logreg, param_grid, cv=5)

    # Fit it to the training data
    logreg_cv.fit(X_train, y_train)

    # Print the optimal parameters and best score
    print("Tuned Logistic Regression Parameter: {}".format(logreg_cv.best_params_))
    print("Tuned Logistic Regression Accuracy: {}".format(logreg_cv.best_score_))

Hold-out set in practice II: Regression

In [31]:
if bool_recomp == True: 
    # Create train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

    # Create the hyperparameter grid
    l1_space = np.linspace(0, 1, 30)
    param_grid = {'l1_ratio': l1_space}

    # Instantiate the ElasticNet regressor: elastic_net
    elastic_net = ElasticNet()

    # Setup the GridSearchCV object: gm_cv
    gm_cv = GridSearchCV(elastic_net, param_grid, cv=5)

    # Fit it to the training data
    gm_cv.fit(X_train, y_train)

    # Predict on the test set and compute metrics
    y_pred = gm_cv.predict(X_test)
    r2 = gm_cv.score(X_test, y_test)
    mse = mean_squared_error(y_test, y_pred)
    print("Tuned ElasticNet l1 ratio: {}".format(gm_cv.best_params_))
    print("Tuned ElasticNet R squared: {}".format(r2))
    print("Tuned ElasticNet MSE: {}".format(mse))

## Preprocessing and pipelines

Exploring categorical features - Boxplots are particularly useful for visualizing categorical features

In [32]:
if bool_recomp == True: 
    # Read 'gapminder.csv' into a DataFrame: df
    df = pd.read_csv('gapminder.csv')

    # Create a boxplot of life expectancy per region
    df.boxplot(column='life', by='Region', rot=60)

    # Show the plot
    plt.show()

Creating dummy variables

In [33]:
if bool_recomp == True: 
    # Create dummy variables: df_region
    df_region = pd.get_dummies(df)

    # Print the columns of df_region
    print(df_region.columns)

    # Create dummy variables with drop_first=True: df_region
    df_region = pd.get_dummies(df, drop_first=True)

    # Print the new columns of df_region
    print(df_region.columns)

Regression with categorical features

In [34]:
if bool_recomp == True: 
    # Instantiate a ridge regressor: ridge
    ridge = Ridge(alpha=0.5, normalize=True)

    # Perform 5-fold cross-validation: ridge_cv
    ridge_cv = cross_val_score(ridge, X, y, cv=5)

    # Print the cross-validated scores
    print(ridge_cv)

Dropping missing data

In [35]:
if bool_recomp == True: 
    # Convert '?' to NaN
    df[df == '?'] = np.nan

    # Print the number of NaNs
    print(df.isnull().sum())

    # Print shape of original DataFrame
    print("Shape of Original DataFrame: {}".format(df.shape))

    # Drop missing values and print shape of new DataFrame
    df = df.dropna()

    # Print shape of new DataFrame
    print("Shape of DataFrame After Dropping All Rows with Missing Values: {}".format(df.shape))

Imputing missing data in a ML Pipeline I

In [36]:
if bool_recomp == True:
    # Setup the Imputation transformer: imp
    imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)

    # Instantiate the SVC classifier: clf
    clf = SVC()

    # Setup the pipeline with the required steps: steps
    steps = [('imputation', imp),
            ('SVM', clf)]

Imputing missing data in a ML Pipeline II

In [37]:
if bool_recomp == True:
    # Setup the pipeline steps: steps
    steps = [('imputation', Imputer(missing_values='NaN', strategy='most_frequent', axis=0)),
            ('SVM', SVC())]

    # Create the pipeline: pipeline
    pipeline = Pipeline(steps)

    # Create training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Fit the pipeline to the train set
    pipeline.fit(X_train, y_train)

    # Predict the labels of the test set
    y_pred = pipeline.predict(X_test)

    # Compute metrics
    print(classification_report(y_test, y_pred))

Centering and scaling your data

In [38]:
if bool_recomp == True:
    # Scale the features: X_scaled
    X_scaled = scale(X)

    # Print the mean and standard deviation of the unscaled features
    print("Mean of Unscaled Features: {}".format(np.mean(X))) 
    print("Standard Deviation of Unscaled Features: {}".format(np.std(X)))

    # Print the mean and standard deviation of the scaled features
    print("Mean of Scaled Features: {}".format(np.mean(X_scaled))) 
    print("Standard Deviation of Scaled Features: {}".format(np.std(X_scaled)))

Centering and scaling in a pipeline

In [39]:
if bool_recomp == True:
    # Setup the pipeline steps: steps
    steps = [('scaler', StandardScaler()),
            ('knn', KNeighborsClassifier())]

    # Create the pipeline: pipeline
    pipeline = Pipeline(steps)

    # Create train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Fit the pipeline to the training set: knn_scaled
    knn_scaled = pipeline.fit(X_train, y_train)

    # Instantiate and fit a k-NN classifier to the unscaled data
    knn_unscaled = KNeighborsClassifier().fit(X_train, y_train)

    # Compute and print metrics
    print('Accuracy with Scaling: {}'.format(knn_scaled.score(X_test, y_test)))
    print('Accuracy without Scaling: {}'.format(knn_unscaled.score(X_test, y_test)))

Bringing it all together I: Pipeline for classification

In [40]:
if bool_recomp == True:
    # Setup the pipeline
    steps = [('scaler', StandardScaler()),
             ('SVM', SVC())]

    pipeline = Pipeline(steps)

    # Specify the hyperparameter space
    parameters = {'SVM__C':[1, 10, 100],
                  'SVM__gamma':[0.1, 0.01]}

    # Create train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

    # Instantiate the GridSearchCV object: cv
    cv = GridSearchCV(pipeline, param_grid=parameters)

    # Fit to the training set
    cv.fit(X_train, y_train)

    # Predict the labels of the test set: y_pred
    y_pred = cv.predict(X_test)

    # Compute and print metrics
    print("Accuracy: {}".format(cv.score(X_test, y_test)))
    print(classification_report(y_test, y_pred))
    print("Tuned Model Parameters: {}".format(cv.best_params_))

Bringing it all together II: Pipeline for regression

In [41]:
if bool_recomp == True:
    # Setup the pipeline steps: steps
    steps = [('imputation', Imputer(missing_values='NaN', strategy='mean', axis=0)),
             ('scaler', StandardScaler()),
             ('elasticnet', ElasticNet())]

    # Create the pipeline: pipeline 
    pipeline = Pipeline(steps)

    # Specify the hyperparameter space
    parameters = {'elasticnet__l1_ratio': np.linspace(0,1,30)}

    # Create train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

    # Create the GridSearchCV object: gm_cv
    gm_cv = GridSearchCV(pipeline, param_grid=parameters)

    # Fit to the training set
    gm_cv.fit(X_train, y_train)

    # Compute and print the metrics
    r2 = gm_cv.score(X_test, y_test)
    print("Tuned ElasticNet Alpha: {}".format(gm_cv.best_params_))
    print("Tuned ElasticNet R squared: {}".format(r2))